In [8]:
search_for = 76
start_from = 0
threads = 1
thread = 0
log_freq = 100000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.005990266799926758
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 89552211
type: [1, 1, 1, 1, 76] 76
cases of this type: 33362176
100000 46.50172161675962
200000 48.894578595297745
300000 36.018874313279035
400000 46.176285575346824
500000 36.37404321569446
600000 39.58317549205266
700000 30.63684751475623
800000 38.88141795992607
900000 30.210182171908283
1000000 36.52654773420065
1100000 33.17617711935507
1200000 32.870134894709324
1300000 36.61141294826946
1400000 28.611009279780543
1500000 35.87632525319159
1600000 28.116080691289874
1700000 35.3339939238281
1800000 26.23298921301266
1900000 36.798359858635955
2000000 28.295910154545627
2100000 36.0444978189795
2200000 31.54649707252111
2300000 30.62620610554526
2400000 35.73385845673474
2500000 26.99788398159349
2600000 34.611682745979465
2700000 27.339795242717127
2800000 35.06986597303376
2900000 26.640233064931962
3000000 34.7552545003

29500000 15.391086241481927
29600000 15.413972044382133
29700000 13.273963588442848
29800000 17.449945892724738
29900000 16.52230308640443
30000000 17.093895110867088
30100000 13.96456648205494
30200000 18.18477259781953
30300000 16.713276864761923
30400000 17.599413257897677
30500000 16.22285761993
30600000 14.986340564547122
30700000 16.059465631883118
30800000 16.978668981174973
30900000 19.48252495245194
31000000 12.545020469277107
31100000 17.10209177853317
31200000 14.489440703705622
31300000 15.697703389742454
31400000 14.478696264205873
31500000 16.596741241777732
31600000 16.258076854422264
31700000 15.804062709075767
31800000 17.475986247078705
31900000 13.083178421451393
32000000 16.250275433709625
32100000 12.130011126623321
32200000 18.10248729109419
32300000 15.352077385312773
32400000 18.447127444011677
32500000 14.493662107994549
32600000 14.609740152097295
32700000 16.33541624826119
32800000 14.196060003397605
32900000 18.021145088642964
33000000 13.9685919709804
33100

58900000 5.410882076690304
59000000 5.360867124099199
59100000 4.879094766136896
59200000 4.936929489535589
59300000 5.271558282778969
59400000 5.461120339511873
59500000 4.693657320104486
59600000 4.8050328681785395
59700000 4.748352389536274
59800000 5.802063822475793
59900000 4.985452995046763
60000000 4.584506738708567
60100000 4.845855127806344
best so far: 0
type: [1, 1, 1, 38, 2] 76
cases of this type: 877952
60200000 2.940998722191353
60300000 1.9901539083406687
60400000 1.9179080595415499
60500000 1.8919961250822968
60600000 1.9118310811657788
60700000 1.9626925831785902
60800000 1.9057867865692697
60900000 1.9204077208104908
61000000 1.8495116359010373
best so far: 0
type: [1, 1, 1, 76, 1] 76
cases of this type: 438976
61100000 1.8742749024554426
61200000 1.8576153030090392
61300000 1.8105630772228014
61400000 1.875372196376172
best so far: 0
type: [1, 1, 2, 1, 38] 76
cases of this type: 8340544
61500000 4.265329368309591
61600000 6.950658992260607
61700000 7.497674197003044


84100000 0.3860618646127006
84200000 0.3998268173175004
84300000 0.4543163747745633
84400000 0.7043471441401389
84500000 0.6694437286540091
84600000 0.6807571420662165
best so far: 0
type: [1, 2, 2, 19, 1] 76
cases of this type: 54872
84700000 0.4911548939929796
best so far: 0
type: [1, 2, 19, 1, 2] 76
cases of this type: 11552
best so far: 0
type: [1, 2, 19, 2, 1] 76
cases of this type: 5776
best so far: 0
type: [1, 2, 38, 1, 1] 76
cases of this type: 2888
best so far: 0
type: [1, 4, 1, 1, 19] 76
cases of this type: 521284
84800000 0.5576573015695142
84900000 0.5935985519862692
85000000 0.38472416468080156
85100000 0.49419030862913793
85200000 0.5649209674335837
best so far: 0
type: [1, 4, 1, 19, 1] 76
cases of this type: 27436
best so far: 0
type: [1, 4, 19, 1, 1] 76
cases of this type: 1444
best so far: 0
type: [1, 19, 1, 1, 4] 76
cases of this type: 4864
best so far: 0
type: [1, 19, 1, 2, 2] 76
cases of this type: 2432
best so far: 0
type: [1, 19, 1, 4, 1] 76
cases of this type: 12